# ElasticSearch & Python
---



<h3>Ankit Maheshwari</h3>
<br/><br/><br/>
<div style="text-align: right"> 

Twitter: @ankitind <br/>
Web: www.betout.com 

</div>


# 1. What is Elasticsearch
---
- Full-text Search Engine
- NoSQL Database
- Analytics Engine
- Lucene based
- Schemaless
- RESTful interface
- Inverted Indices
- (Nearly) Real time
- ELK Stack (Elasticsearch, Kibana, Logstash)

# 2. Installing ElasticSearch
---
- Install via docker
- Create a directory to store ES Data
- Start ELK Stack

> docker pull qnib/elk <br>
> mkdir -p ~/data <br>
> docker run -d --name elasticsearch -v ~/data:/usr/share/elasticsearch/data -p 9200:9200 elasticsearch
docker container run -d --name elasticsearch -v ~/data:/usr/share/elasticsearch/data -p 9200:9200 qnib/elk

> sudo docker run -d -v $(pwd):/usr/share/elasticsearch/data -p 5601:5601 -p 9200:9200 -p 5044:5044 -it --name elk sebp/elk

Test the stack <br>
ElasticsSearch http://localhost:9200 & Kibana http://localhost:5601

# 3. Comparison with a RDBMS 
### like MySQL/Oracle

| Relational DBs | Elasticsearch                         |
|:---------------|:--------------------------------------|
| Database       | Index                                 |
| Partition      | Shard                                 |
| Table          | Type                                  |
| Row            | Document                              |
| Column         | Field                                 |
| Schema         | Mapping                               |
| Index          | Everything is <br> already Indexed :) |
| SQL            | DSL <br> Domain Specific Language     |



### 3.2 JSON schema cheatsheet / quick reference

| Javascript     | Python                                | Example                    |
|:---------------|:--------------------------------------|:---------------------------|
| string         | string                                | "Name"                     |
| number         | int/float                             | 42                         |
| object         | dict (dictionary)                     | {"name": "sam", "age": 26} |
| array          | list                                  | ["foo","bar", 5,"hello"]   |
| boolean        | bool                                  | tr                         |



3.2 Installing Python libraries 

In [4]:
#Few libraries we will using 
#!pip install ujson requests ipywidgets 
#!pip install -U certifi
#!pip install -U elasticsearch 
#!pip install faker elasticsearch_dsl
import faker
import requests
import ujson as json
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search, DocType, Date, Integer, Keyword, Text
from datetime import datetime
from elasticsearch_dsl.connections import connections
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Dropdown
from IPython.display import clear_output


ES_HOST = 'http://team.ankit.one:9200'
INDEX ='umbrellacorp'
es = Elasticsearch(ES_HOST)
print(es)


<Elasticsearch([{'port': 9200, 'host': 'team.ankit.one'}])>


Print all indices of Elasticsearch

In [6]:
# list all the indexes
indices=es.indices.get_alias().keys()
sorted(indices)

[]

# 4. Mapping
Mapping is the process of defining how a document, and the fields it contains, are stored and indexed. 

#### Mapping Types
Each index has one or more mapping types, which are used to divide the documents in an index into logical groups. 
>User documents might be stored in a user type, and blog posts in a blogpost type.

##### Meta-fields
>Meta-fields are used to customize how a document’s metadata associated is treated. 
Examples of meta-fields include the document’s _index, _type, _id, and _source fields.

##### Fields or properties
>Each mapping type contains a list of fields or properties pertinent to that type. 
A user type might contain title, name, and age fields

```
PUT my_index 
{"mappings": {
    "user": { 
      "_all":       { "enabled": false  }, 
      "properties": {
        "name":     { "type": "text"  }, 
        "age":      { "type": "integer" }  
              },//end of user
    "blogpost": { 
      "_all":       { "enabled": false  }, 
      "properties": { 
        "title":    { "type": "text"  }, 
        "body":     { "type": "text"  }, 
        "user_id":  { "type": "keyword" },
        "created":  { "type":   "date", 
                      "format": "strict_date" } 
                 } // end of blogpost
      } //end of mappings
}```

# 5. Query and filter contex

### Query context
This query answers the question “How well does this document match this query clause?” Besides deciding whether or not the document matches, the query clause also calculates a _score representing how well the document matches

### Filter context
In filter context, a query clause answers the question “Does this document match this query clause?” The answer is a simple Yes or No — no scores are calculated. Filter context is mostly used for filtering structured data, e.g.
Does this timestamp fall into the range 2015 to 2016? Is the status field set to "published"?

## 5.1 Filter query
- When working with exact values, we will be working with non-scoring, filtering queries. 
- Filters are important because they are very fast. 
- They do not calculate relevance (avoiding the entire scoring phase) and are easily cached.
- We use a constant_score to convert the term query into a filter


In [38]:
myquery = {"query":  {"constant_score" : {"filter" : {"term" : {"country":"Israel"}}}}}
res = es.search(index=INDEX, body=myquery)
for items in res['hits']['hits']:
    print(items)
    print(items['_source']['name'] + " (Id: " + items['_id'] + ") has a score of: " + str(items['_score']))

## 6. Types of Queries
Leaf Queries
- Match all
- Full Text
- Term Level
>"query" : {"queryType" : {"fieldname" : "fieldvalue"}}

Compund Queries
- Bool Query
- Constant Score Queries




### 6.1 Match all
The most simple query, which matches all documents, giving them all a _score of 1.0.

In [39]:
# save match all query as python variable
myquery={"query": 
         {"match_all": {}}
        }

# execute the query using body parameter and return total number of records
# select count(*) from table
res = es.search(index=INDEX, body=myquery)  

print("Total records found: {rec}".format(rec=res['hits']['total']))
for x in range(0, res['hits']['total']):
    print("\n" + str(x+1))
    for key, value in res['hits']['hits'][x]['_source'].items():
        print(str(key) + ": " + str(value))
    if x == 1:
        print("-- breaking--")
        break 

Total records found: 50002

1
email: sethmatthews@example.org
job: Sports coach
total_discount_revenue: 58.68
revenue: 763
rfm_score: 21
email_unsubscribe: False
signup_date: 2017-04-27T21:22:06
avg_revenue_per_month: 326.0
address: 240 Robert Parkway Apt. 656
Port Marissa, AR 87861-5159
r_score: 3
lifecycle: 7
country: Saint Barthelemy
f_score: 9
num_of_orders: 10.0
m_score: 2
city: West Douglas
fav_color: Gray
name: Jacob Lewis
discount_percentage: 29

2
email: chill@example.com
job: Higher education careers adviser
total_discount_revenue: 58.68
revenue: 4602
rfm_score: 21
email_unsubscribe: False
signup_date: 2017-03-17T11:26:56
avg_revenue_per_month: 195.6
address: 12953 James Ferry
Moranland, SD 96287
r_score: 8
lifecycle: 1
country: Israel
f_score: 9
num_of_orders: 10.0
m_score: 6
city: Smithville
fav_color: Fuchsia
name: Jamie Faulkner
discount_percentage: 21
-- breaking--


In [45]:
##ALso defining field variable to be used to get all field names in susequents slides
field=[]
res = requests.get(ES_HOST + '/umbrellacorp/user/_mapping')
for a in res.json()[INDEX]['mappings']['user']['properties'].keys():
    field.append(a)
def func(Query_type):
    myquery={"query":{Query_type: {}}}
    res = es.search(index=INDEX, body=myquery) 
    print("Total records found: {rec}".format(rec=res['hits']['total']))
    for x in range(0, res['hits']['total']):
        print("\n" + str(x+1))
        for key, value in res['hits']['hits'][x]['_source'].items():
            print(str(key) + ": " + str(value))
        if x == 1:
            print("-- breaking--")
            break 


interact_manual(func,  Query_type={'Get All Records':'match_all'});


### 6.2 Full text queries


<img src="https://qbox.io/img/blog/elasticsearch-queries-example.png">

### 6.2.1 Match Query
- match queries accept text/numerics/dates, analyzes them, and constructs a query.
- matches even if one term is match
- more the terms better the score

In [48]:
myquery={
    "query": {
        "match" : {"name" : "Jamie Faulkner"}
    }}
res = es.search(index=INDEX, body=myquery) 
print("Total records found: {rec}".format(rec=res['hits']['total']))
for x in range(0, res['hits']['total']):
    print("\n" + str(x+1))
    for key, value in res['hits']['hits'][x]['_source'].items():
        print(str(key) + ": " + str(value))
    if x == 1:
        print("-- breaking--")
        break 

Total records found: 176

1
email: chill@example.com
job: Higher education careers adviser
total_discount_revenue: 58.68
revenue: 4602
rfm_score: 21
email_unsubscribe: False
signup_date: 2017-03-17T11:26:56
avg_revenue_per_month: 195.6
address: 12953 James Ferry
Moranland, SD 96287
r_score: 8
lifecycle: 1
country: Israel
f_score: 9
num_of_orders: 10.0
m_score: 6
city: Smithville
fav_color: Fuchsia
name: Jamie Faulkner
discount_percentage: 21

2
email: grossalejandro@example.org
job: Newspaper journalist
total_discount_revenue: 58.68
revenue: 8764
rfm_score: 21
email_unsubscribe: True
signup_date: 2017-02-28T04:53:17
avg_revenue_per_month: 217.33333333333334
address: 8354 Sergio Roads Apt. 426
South Christopher, MP 40641-0898
r_score: 3
lifecycle: 9
country: Singapore
f_score: 9
num_of_orders: 10.0
m_score: 5
city: Andrewland
fav_color: GoldenRod
name: Jacob Faulkner
discount_percentage: 12
-- breaking--


In [49]:
def func(Query_type, Dimension, Value):
    myquery={
    "query": {
        Query_type : {Dimension : Value}
    }}
    res = es.search(index=INDEX, body=myquery) 
    print("Total records found: {rec}".format(rec=res['hits']['total']))
    for x in range(0, res['hits']['total']):
        print("\n" + str(x+1))
        for key, value in res['hits']['hits'][x]['_source'].items():
            print(str(key) + ": " + str(value))
        if x == 1:
            print("-- breaking--")
            break 


interact_manual(func,  Query_type={'Match':'match'}, Dimension = field, Value = '');

### 6.2.2 "Match Phrase"  (match_phrase) Query
The match_phrase query analyzes the text and only results if terms come in same order

In [51]:
myquery={ "query": {
        "match_phrase" : { "name" : "Samuel"}
     }}
res = es.search(index=INDEX, body=myquery) 
print("Total records found: {rec}".format(rec=res['hits']['total']))
for x in range(0, res['hits']['total']):
    print("\n" + str(x+1))
    for key, value in res['hits']['hits'][x]['_source'].items():
        print(str(key) + ": " + str(value))
    if x == 1:
        print("-- breaking--")
        break 

Total records found: 117

1
email: jamescruz@example.net
job: Scientist, physiological
total_discount_revenue: 58.68
revenue: 1237
rfm_score: 21
email_unsubscribe: True
signup_date: 2017-01-09T13:56:41
avg_revenue_per_month: 652.0
address: 208 Chapman Haven Suite 391
Gutierrezview, ID 79514
r_score: 2
lifecycle: 5
country: Russian Federation
f_score: 8
num_of_orders: 10.0
m_score: 5
city: Scottville
fav_color: ForestGreen
name: Samuel Gonzalez
discount_percentage: 13

2
email: jack89@example.org
job: Ambulance person
total_discount_revenue: 58.68
revenue: 4167
rfm_score: 21
email_unsubscribe: False
signup_date: 2017-05-14T13:39:12
avg_revenue_per_month: 978.0
address: 64358 James Manor Suite 220
Smithside, NJ 61078
r_score: 1
lifecycle: 6
country: Bolivia
f_score: 3
num_of_orders: 10.0
m_score: 6
city: Burtonmouth
fav_color: LightCyan
name: Samuel Peterson
discount_percentage: 21
-- breaking--


In [360]:
def func(Query_type, Dimension, Value):
    myquery={
    "query": {
        Query_type : {Dimension : Value}
    }}
    print(myquery)
    res = es.search(index="titanic", body=myquery) 
    print("Total records found: {rec}".format(rec=res['hits']['total']))
    for x in range(0, res['hits']['total']):
        print("\n" + str(x+1))
        for key, value in res['hits']['hits'][x]['_source'].items():
            print(str(key) + ": " + str(value))
        if x == 1:
            print("-- breaking--")
            break 


interact_manual(func,  Query_type={'Match':'match', 'Match Phrase':'match_phrase'}, Dimension = field, Value = '');

### 6.2.3 Term Query
While the full text queries will analyze the query string before executing, the term-level queries operate on the **exact terms** that are stored in the inverted index.

These queries are usually used for structured data like numbers, dates, and enums, rather than full text fields.

In [55]:
myquery={"query": {"term" : { "fav_color" : "red" }  }}

res = es.search(index=INDEX, body=myquery) 
for itemkey, itemvalue in res['hits']['hits'][0]["_source"].items():
    print(str(itemkey) + ": " + str(itemvalue))

email: madeline97@example.net
job: Logistics and distribution manager
total_discount_revenue: 58.68
revenue: 6343
rfm_score: 21
email_unsubscribe: True
signup_date: 2017-02-01T07:43:05
avg_revenue_per_month: 326.0
address: 415 Castro Pike Suite 945
Wardhaven, VA 16587-3879
r_score: 5
lifecycle: 6
country: British Virgin Islands
f_score: 7
num_of_orders: 10.0
m_score: 9
city: New Dawnborough
fav_color: Red
name: Kelli Maxwell
discount_percentage: 3


In [56]:
def func(Query_type, Dimension, Value):
    myquery={
    "query": {
        Query_type : {Dimension : Value}
    }}
    res = es.search(index=INDEX, body=myquery) 
    print("Total records found: {rec}".format(rec=res['hits']['total']))
    for x in range(0, res['hits']['total']):
        print("\n" + str(x+1))
        for key, value in res['hits']['hits'][x]['_source'].items():
            print(str(key) + ": " + str(value))
        if x == 1:
            print("-- breaking--")
            break 


interact_manual(func,  Query_type={'Match':'match', 'Match Phrase':'match_phrase', 'Absolute Phrase':'term'}, Dimension = field, Value = '');

### 6.2.4 Terms query
Filters documents that have fields that match any of the provided terms (not analyzed).
>The terms query is also aliased with **in** as in MySQL

In [70]:
myquery={"query": { "terms" : { "country" : ['Vanuatu', 'Greenland', 'Mexico']}}}
res = es.search(index=INDEX,  body=myquery)
print("Total records found - {a}".format(a=(res['hits']['total'])))
#print("Third record : {a}".format(a=(res['hits']['hits'][0]['_source'])))

Total records found - 0


### 6.2.5 Range Query
Matches documents with fields that have terms within a certain range. 


In [125]:
myquery = {"query" : {"range": {"PassengerId":{"gte":990,"lte":1000}}}}
res = es.search(index=INDEX,  body=myquery)
print("Total records found - {a}".format(a=res['hits']['total']))

myqueryTimeExample = {"query" : {"range":{"timestamp":{"gte":"2015-01-01 00:00:00", "lte":"now"}}}}


Total records found - 11


In [71]:
def func(Query_type, Dimension,  Less_Than, Greater_Than):
    myquery = {"query" : {Query_type: {Dimension:{"gte":Greater_Than,"lte":Less_Than}}}}
    res = es.search(index=INDEX, body=myquery) 
    print("Total records found: {rec}".format(rec=res['hits']['total']))
    for x in range(0, res['hits']['total']):
        print("\n" + str(x+1))
        for key, value in res['hits']['hits'][x]['_source'].items():
            print(str(key) + ": " + str(value))
        if x == 1:
            print("-- breaking--")
            break 


interact_manual(func,  Query_type='range', Dimension = field,   Greater_Than = '', Less_Than = '');

In [72]:
## Summary of Leaf Queries
# 1. Match_all
q1 = {"query" : {"match_all":{}}}

# 2. Match
q2 = {"query" : {"match":{"Sex":"male"}}}

# 3. Match Phrase
q3 = {"query" : {"match_phrase":{"Sex":"male"}}}

# 4. Term
q4 = {"query" : {"term":{"Sex":"male"}}}

#5. Terms
q5 = {"query" : {"terms":{"Sex":["male", "female"]}}}

#6. Range
q6 = {"query" : {"range" : {"PassengerId" : {"gte" : 1000, "lte" : 2000}}}}

res = es.search(index=INDEX, body=q5)
print(res['hits']['total'])


0


## 7. Compound queries
Compound queries wrap other compound or leaf queries, either to combine their results and scores, to change their behaviour, or to switch from query to filter context.

# 7.1 Boolean Querey
A query that matches documents matching boolean combinations of other queries.
bool query is composed of four sections:
```{
   "bool" : {
      "must" :     [],
      "should" :   [],
      "must_not" : [],
      "filter":    []
   }}```

In [10]:
myquery = {"query":
               {"bool":{
                    "filter":{"term":{"Age":"22"}},
                    "must":{"term":{"Pclass":"2"}},
                    "must_not":{"term":{"Sex":"female"}}
                    }}}



res = es.count(index="titanic", body=myquery)
print("Total count is: {a}".format(a=res['count']))

Total count is: 2


In [73]:
def func(Query_type, subQuery_type, Dimension,  Value):
    myquery = {"query":{Query_type:{subQuery_type:{"term":{Dimension:Value}}}}}
    res = es.search(index=INDEX, body=myquery) 
    print(myquery)
    print("Total records found: {rec}".format(rec=res['hits']['total']))
    for x in range(0, res['hits']['total']):
        print("\n" + str(x+1))
        for key, value in res['hits']['hits'][x]['_source'].items():
            print(str(key) + ": " + str(value))
        if x == 1:
            print("-- breaking--")
            break 

           
i =interact_manual(func,  
                Query_type={'Compound':'bool'},
                subQuery_type ={'Filter':'filter', 'Must have':'must', 'Must NOT have':'must_not', 'Maybe Have':'should'},
                Dimension = field,   
                Value = ''
               )
display(i)

NameError: name 'display' is not defined